In [27]:
# class 0에서 랜덤하게 1100개를 선택해서 사용 
import csv
import numpy as np
from tabulate import tabulate
from util import util
from classification import classification
from rule_gen import rule_generation
import timeit
from sklearn.metrics import classification_report
import copy
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
import random
from mlxtend.frequent_patterns import apriori

In [44]:
with open('../data/dataset.csv', 'r') as file:
        data_set = [list(filter(None, row)) for row in csv.reader(file)]
        
print(len(data_set))

4615


In [2]:
te = TransactionEncoder()
te_ary = te.fit_transform(data_set)
m_transactions = pd.DataFrame(te_ary, columns=te.columns_)

print(m_transactions.shape)

NameError: name 'TransactionEncoder' is not defined

In [1]:
m_transactions

NameError: name 'm_transactions' is not defined

In [46]:
transactions_0 = pd.DataFrame(
            m_transactions[m_transactions['0']].reset_index(drop=True).drop(['1', '0'], axis=1))
transactions_1 = pd.DataFrame(
            m_transactions[m_transactions['1']].reset_index(drop=True).drop(['1', '0'], axis=1))


In [52]:
random.seed(10)
indices = list( range(0, len(transactions_0)) )
random.shuffle(indices)
transactions_te_0 = transactions_0.iloc[indices[:417],:]
#transactions_tr_00 = transactions_0.iloc[indices[417:810],:]
transactions_tr_0 = transactions_0.iloc[indices[417:],:]

indices = list( range(0, len(transactions_1)) )
random.shuffle(indices)
transactions_te_1 = transactions_1.iloc[indices[:43],:]
transactions_tr_1 = transactions_1.iloc[indices[43:],:]

tr_0_ary=(transactions_tr_0.values).astype('int')
tr_1_ary=(transactions_tr_1.values).astype('int')

#tr_00_ary=(transactions_tr_00.values).astype('int')

transactions_tr = pd.concat([transactions_tr_0, transactions_tr_1])

In [53]:
frequent_items = apriori(transactions_tr, min_support=0.1)
print(transactions_tr.shape, len(frequent_items))

(4155, 147) 22554


In [54]:
natt = transactions_tr.shape[1]  #number of attributes 
nf = len(frequent_items)   #number of frequent items 
a = frequent_items['itemsets']
b = [list(x) for x in a]
c= np.zeros((natt,nf))
for i in range(len(frequent_items)):
    c[b[i],i]=1

In [61]:
#lift
newc = copy.deepcopy(c)
x_00 = np.matmul(tr_0_ary, c)
x_01 = np.matmul(tr_1_ary, c)

item_len = np.sum(c, axis=0) #item length
cls0 = np.zeros(nf)   #frequent_items['support']
cls1 = np.zeros(nf) 
p = np.zeros(nf)
conf = np.zeros(nf)
indc = np.zeros(nf, dtype=int) 
cnt=0
for i in range(x_00.shape[1]):
    cls0[i] = (x_00[:,i]>=item_len[i]).sum()/x_00.shape[0]
    cls1[i] = (x_01[:,i]>=item_len[i]).sum()/x_01.shape[0]
    p[i] = ((x_00[:,i]>=item_len[i]).sum()+ (x_01[:,i]>=item_len[i]).sum())/(x_00.shape[0]+x_01.shape[0])
    if cls0[i]/p[i]>1:
        indc[i] = 0
        conf[i] = cls0[i]/p[i]
        cnt = cnt +1
    elif cls0[i]/p[i]<1 and cls1[i]/p[i]>1:
        indc[i] = 1
        conf[i] = cls1[i]/p[i]
        cnt = cnt +1
    else:
        conf[i]=0
        newc[:,i]=0
        indc[i] =-1
print('class rules', cnt)



class rules 22554


In [63]:
#1-ccs/cls
newc = copy.deepcopy(c)
x_00 = np.matmul(tr_0_ary, c)
x_01 = np.matmul(tr_1_ary, c)

item_len = np.sum(c, axis=0) #item length
cls0 = np.zeros(nf)   #frequent_items['support']
cls1 = np.zeros(nf) 
p = np.zeros(nf)
conf = np.zeros(nf)
indc = np.zeros(nf, dtype=int) 
cnt=0
for i in range(x_00.shape[1]):
    cls0[i] = (x_00[:,i]>=item_len[i]).sum()/x_00.shape[0]
    cls1[i] = (x_01[:,i]>=item_len[i]).sum()/x_01.shape[0]
    p[i] = ((x_00[:,i]>=item_len[i]).sum()+ (x_01[:,i]>=item_len[i]).sum())/(x_00.shape[0]+x_01.shape[0])
    if cls1[i]/cls0[i]<1:
        indc[i] = 0
        conf[i] = 1- cls1[i]/cls0[i]
        cnt = cnt +1
    elif cls1[i]/cls0[i]>1:
        indc[i] = 1
        conf[i] = 1- cls0[i]/cls1[i]
        cnt = cnt +1
    else:
        conf[i]=0
        newc[:,i]=0
        indc[i] =-1
print('class rules', cnt)


class rules 22554


In [64]:
from sklearn.metrics import roc_auc_score

print((indc==0).sum(), (indc==1).sum())
print(conf[indc==0].mean(), conf[indc==1].mean())

te_0_ary=(transactions_te_0.values).astype('int')
te_1_ary=(transactions_te_1.values).astype('int')

#first rule used
y_0 = np.matmul(te_0_ary, newc)
y_1 = np.matmul(te_1_ary, newc)

pred = np.zeros((y_0.shape[0]+y_1.shape[0],2))
cnt0=0
cnt1=0
for i in range(y_0.shape[0]):
    maxp=-1
    for j in range(y_0.shape[1]):
        if (indc[j]==0 and y_0[i,j]>=item_len[j]):
            maxp = max(maxp, conf[j])
    if (maxp==-1):
        cnt0 = cnt0+1    
    else: 
        pred[i,0] = maxp
        
    maxp=-1
    for j in range(y_0.shape[1]):
        if (indc[j]==1 and y_0[i,j]>=item_len[j]):
            maxp = max(maxp, conf[j])
           
    if (maxp==-1):
        cnt1 = cnt1+1    
    else: 
        pred[i,1] = maxp
print(y_0.shape[0],cnt0, cnt1)

cnt0=0
cnt1=0
for i in range(y_1.shape[0]):
    maxp=-1
    for j in range(y_1.shape[1]):
        if (indc[j]==0 and y_1[i,j]>=item_len[j]):
            maxp = max(maxp, conf[j])
    if (maxp==-1):
        cnt0 = cnt0+1    
    else: 
        pred[i+y_0.shape[0],0] = maxp
        
    maxp=-1
    for j in range(y_1.shape[1]):
        if (indc[j]==1 and y_1[i,j]>=item_len[j]):
            maxp = max(maxp, conf[j])
           
    if (maxp==-1):
        cnt1 = cnt1+1    
    else: 
        pred[i+y_0.shape[0],1] = maxp
        
print(y_1.shape[0],cnt0, cnt1)

y = np.concatenate((np.zeros(te_0_ary.shape[0]), np.ones(te_1_ary.shape[0])), axis=0)
auc = roc_auc_score(y,pred[:,1])
print(auc)
y = np.concatenate((np.zeros(te_0_ary.shape[0]), np.ones(te_1_ary.shape[0])), axis=0)
auc = roc_auc_score(y,-pred[:,0])
print(auc)

15179 7375
0.5016779539597047 0.31930184734930256
417 3 3
43 3 0
0.856198761920696
0.7621158886844013


In [65]:
from sklearn.metrics import roc_auc_score

print((indc==0).sum(), (indc==1).sum())
print(conf[indc==0].mean(), conf[indc==1].mean())

te_0_ary=(transactions_te_0.values).astype('int')
te_1_ary=(transactions_te_1.values).astype('int')

#all matched rules  used
y_0 = np.matmul(te_0_ary, newc)
y_1 = np.matmul(te_1_ary, newc)

pred = np.zeros((y_0.shape[0]+y_1.shape[0],2))
cnt0=0
cnt1=0
        
for i in range(y_0.shape[0]):
    cnt=0
    for j in range(y_0.shape[1]):
        if (indc[j]==0 and y_0[i,j]>=item_len[j]):
            pred[i,0] = pred[i,0] + conf[j]
            cnt = cnt+1
    if (cnt==0):
        cnt0 = cnt0+1    
    else: 
        pred[i,0] = pred[i,0]/cnt
        
    cnt=0
    for j in range(y_0.shape[1]):
        if (indc[j]==1 and y_0[i,j]>=item_len[j]):
            pred[i,1] = pred[i,1] + conf[j]
            cnt = cnt+1
           
    if (cnt==0):
        cnt1 = cnt1+1    
    else: 
        pred[i,1] = pred[i,1]/cnt
print(y_0.shape[0],cnt0, cnt1)

cnt0=0
cnt1=0
for i in range(y_1.shape[0]):
    cnt=0
    x=0
    for j in range(y_1.shape[1]):
        if (indc[j]==0 and y_1[i,j]>=item_len[j]):
            x = x + conf[j]
            cnt = cnt+1
    if (cnt==0):
        cnt0 = cnt0+1    
    else: 
        pred[i+y_0.shape[0],0] = x/cnt
        
    cnt=0
    x=0
    for j in range(y_1.shape[1]):
        if (indc[j]==1 and y_1[i,j]>=item_len[j]):
            x = x + conf[j]
            cnt = cnt+1
           
    if (cnt==0):
        cnt1 = cnt1+1    
    else: 
        pred[i+y_0.shape[0],1] = x/cnt
        
print(y_1.shape[0],cnt0, cnt1)

y = np.concatenate((np.zeros(te_0_ary.shape[0]), np.ones(te_1_ary.shape[0])), axis=0)
auc = roc_auc_score(y,pred[:,1])
print(auc)
y = np.concatenate((np.zeros(te_0_ary.shape[0]), np.ones(te_1_ary.shape[0])), axis=0)
auc = roc_auc_score(y,-pred[:,0])
print(auc)

15179 7375
0.5016779539597047 0.31930184734930256
417 3 3
43 3 0
0.8729016786570744
0.827254475489376
